# Import libraries

In [0]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras import datasets
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report

# Get File

In [5]:
!pip install PyDrive

    100% |████████████████████████████████| 993kB 7.4MB/s 
  Running setup.py bdist_wheel for PyDrive ... - \ done
  Stored in directory: /content/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [7]:
fileId = drive.CreateFile({'id': '1LzIXsiDblKpLHJugw5Sn7PZ4zkiqDh_L'})
#DRIVE_FILE_ID is file id 
print(fileId['title'])  # folder_data.zip
# Save Drive file as a local file
fileId.GetContentFile('train_cats_and_dogs.zip')  

fileId = drive.CreateFile({'id': '1QINtIZWxm6nd3_HwEl8xjotxU2WWRfxH'})
#DRIVE_FILE_ID is file id 
print(fileId['title'])  # folder_data.zip
# Save Drive file as a local file
fileId.GetContentFile('test_cats_and_dogs.zip')



train.zip
test1.zip


In [8]:
!ls

datalab  test_cats_and_dogs.zip  train_cats_and_dogs.zip


In [0]:
!unzip train_cats_and_dogs.zip -d ./

In [0]:
!unzip test_cats_and_dogs.zip -d ./

In [11]:
!ls

datalab  test1	test_cats_and_dogs.zip	train  train_cats_and_dogs.zip


In [0]:
!cd train

In [0]:
!mkdir cats

In [0]:
!mv cats train

In [0]:
!cd train

In [0]:
!cd train/

In [0]:
!mv train/cat.* train/cats/

In [0]:
!mkdir train/dogs

In [0]:
!mv train/dog.* train/dogs

In [0]:
target_size = (20,20)

In [122]:
idg = ImageDataGenerator(rescale=1./255)
train_image_generator = idg.flow_from_directory(
    'train',classes = ['cats','dogs'],class_mode = 'categorical',
    target_size=target_size,batch_size = 50)

Found 25000 images belonging to 2 classes.


In [123]:
train_image_generator.class_indices

{'cats': 0, 'dogs': 1}

In [124]:
train_image_generator


In [0]:
!mkdir test

In [0]:
!mv test1 test

In [125]:
test_image_generator = idg.flow_from_directory('test',
                                               target_size=target_size,
                                              class_mode='categorical',
                                               batch_size = 50)

Found 12500 images belonging to 1 classes.


In [126]:
test_image_generator.class_indices

{'test1': 0}

# Create Model

In [0]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3), strides = (1,1),
                 activation='relu',input_shape = (20,20,3)))
model.add(Conv2D(16, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(50,activation='relu'))
model.add(Dense(2,activation= 'softmax'))

model.compile(loss = categorical_crossentropy, 
              optimizer = RMSprop(lr = 0.0001, decay=1e-6), 
              metrics = ['accuracy'])

# History

In [0]:
class AccuracyHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))

history = AccuracyHistory()

# Train Model

In [132]:
model.fit_generator(generator=train_image_generator,
                    steps_per_epoch=500, epochs=10,
                   callbacks=[history])

Epoch 1/10
500/500 [==============================] - 515s 1s/step - loss: 0.6493 - acc: 0.6149
Epoch 2/10
288/500 [================>.............] - ETA: 1:10 - loss: 0.6084 - acc: 0.6648

500/500 [==============================] - 194s 389ms/step - loss: 0.6025 - acc: 0.6715
Epoch 3/10
460/500 [==========================>...] - ETA: 30s - loss: 0.5722 - acc: 0.7017

500/500 [==============================] - 402s 804ms/step - loss: 0.5723 - acc: 0.7010
Epoch 4/10
500/500 [==============================] - 794s 2s/step - loss: 0.5473 - acc: 0.7208
Epoch 5/10
 22/500 [>.............................] - ETA: 13:16 - loss: 0.5196 - acc: 0.7309

500/500 [==============================] - 864s 2s/step - loss: 0.5263 - acc: 0.7385
Epoch 6/10
343/500 [===================>..........] - ETA: 4:33 - loss: 0.5091 - acc: 0.7504

500/500 [==============================] - 866s 2s/step - loss: 0.5084 - acc: 0.7505
Epoch 7/10
491/500 [============================>.] - ETA: 15s - loss: 0.4896 - acc: 0.7658

500/500 [==============================] - 863s 2s/step - loss: 0.4890 - acc: 0.7661
Epoch 8/10
500/500 [==============================] - 863s 2s/step - loss: 0.4697 - acc: 0.7775
Epoch 9/10
 37/500 [=>............................] - ETA: 13:13 - loss: 0.4378 - acc: 0.7924

500/500 [==============================] - 812s 2s/step - loss: 0.4516 - acc: 0.7892
Epoch 10/10
349/500 [===================>..........] - ETA: 3:48 - loss: 0.4328 - acc: 0.8021

500/500 [==============================] - 756s 2s/step - loss: 0.4337 - acc: 0.7998


In [0]:
model.save('Dog-Cat_Classifier')

In [0]:
y_pred = model.predict_generator(test_image_generator,steps=250)

In [136]:
print(y_pred.shape)

(12500, 2)


In [0]:
results = [np.argmax(a) for a in y_pred]

In [0]:
np.array(results).tofile('output.csv',sep = '\n')